# 03.04 - Handling Missing Data

This section will cover general considerations for missing data and some Pandas built-in tools to handle them. 

We will refer to missing data as _null_, _NaN_, or _NA_ values.

### Missing Data Conventions

There are generally two approaches in handling missing values:

1. Using a **mask** that globally indicates missing values
2. Choosing a **sentinel value** that indicates a missing entry

### Missing Data in Pandas

Pandas does not have a built-in notion of NA values for non-floating-point data types. Instead, Pandas chose to use **sentinels** for missing data, and further chose to use two already-existing Python null values: the special floating-point <code>NaN</code> value, and the Python <code>None</code> object.  

### <code>None</code>: Pythonic missing data

None is a special Python object, and as such can only be used in arrays with data type <code>'object'</code>:

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

The use of Python objects in an array also means that **performing aggregations will return an error**:

In [3]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### <code>NaN</code>: Missing numerical data

<code>NaN</code>(Not a Number) is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

NumPy supports <code>NaN</code>, which means we can perform the operations using NumPy operators, drastically reducing execution time. 

However, anything involving a <code>NaN</code> object will return <code>NaN</code>:

In [6]:
vals3 = 1 + vals2
vals3.dtype

dtype('float64')

In [7]:
1 + np.nan

nan

In these cases, it is useful to use the nan-ignoring operations:

In [8]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

**Note**: <code>NaN</code> is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

### NaN and None in Pandas

Pandas handles NaN and None interchangeably, even converting between them:

In [9]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point type to accommodate the missing number with NaN:

In [10]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [11]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

More in general, here is the upcasting convention in Pandas:

<pre>
Typeclass    Conversion if NaN   Sentinel Value

floating 	No change 	      np.nan
object 	  No change 	      None or np.nan
integer 	 Cast to float64 	np.nan
boolean 	 Cast to object 	 None or np.nan
</pre>

### Operating on Null Values

There are several useful methods for detecting, removing, and replacing null values in Pandas data structures:  


* <code>isnull()</code>: Generate a boolean mask indicating missing values
* <code>notnull()</code>: Opposite of isnull()
* <code>dropna()</code>: Return a filtered version of the data
* <code>fillna()</code>: Return a copy of the data with missing values filled or imputed

Here are some examples:

#### 1. Detecting null values

We can use a combination of masking + <code>isnull()</code> or <code>notnull()</code> to delete what we don't need:

In [12]:
data = pd.Series([1, np.nan, 'hello', None])

In [13]:
data[data.notnull()]

0        1
2    hello
dtype: object

#### 2. Dropping null values

Instead of deleting them outright, we drop the NaN value directly or use a filling value:

In [14]:
data.dropna()

0        1
2    hello
dtype: object

In [15]:
data.fillna(42)

0        1
1       42
2    hello
3       42
dtype: object

For <code>Dataframe</code>, there are more options since we have to deal with rows/cols: 

In [16]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


By default, <code>dropna()</code> will drop all rows in which any null value is present:

In [17]:
df.dropna()

,0,1,2
1,2.0,3.0,5


We can also drop along a different axis:

In [18]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


However, since these may also drop useful values, it is a good idea to fine tune them with <code>how</code> and <code>thresh</code>.

For example, <code>how='any'</code> will drop the dimension with at least one null value (as by default) and <code>how='all'</code> will drop it only if _all_ the values are null.

In [19]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [20]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


<code>thresh</code> on the other hand, lets you specify a minimum number of non-null values for the row/column to be kept:

In [22]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


#### 3. Filling null values

Instead of dropping the null values altogether, it may be useful to have them replaced with a filling value:

In [24]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill in a single value, like 42:

In [25]:
data.fillna(42)

a     1.0
b    42.0
c     2.0
d    42.0
e     3.0
dtype: float64

We can specify a forward-fill to propagate the previous value forward:

In [26]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or a back-fill to propagate the next values backward:

In [27]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64